### In this Notebook we create a map showing the average grade of schools

In [ ]:
import pandas as pd
import folium as fm

import matplotlib.pyplot as plt
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import openpyxl


import matplotlib as mpl

mpl.style.use('seaborn-whitegrid')

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.titleweight'] = 'bold'

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.facecolor'] = '#f8f8f8'

In [ ]:
#Grades and school information
fun=pd.read_excel('meta_data.xlsx')

In [ ]:
import geopandas as gpd
gdf=gpd.read_file('custom.geo.json')
gdf=gdf[gdf['admin']=='Denmark']

In [ ]:
from shapely.geometry import Point

# Create geographic points for each school location
geo = [Point(xy) for xy in zip(fun['long'], fun['lat'])]

# Create a GeoDataFrame combining the dataframe 'fun' and the geographic points, also setting the coordinate reference system (CRS)
geo_data = gpd.GeoDataFrame(fun, geometry=geo, crs="EPSG:4326")

# Initialize MinMaxScaler for normalization of avg_grade
scaler = MinMaxScaler(feature_range=(1, 2))

# Add a new column 'scaled_values' to the GeoDataFrame by applying MinMaxScaler to the avg_grade
geo_data['scaled_values'] = scaler.fit_transform(geo_data['avg_grade'].values.reshape(-1, 1))

# Set colormap and normalize coloring based on min and max values of avg_grade
cmap = plt.get_cmap('plasma')
norm = mcolors.Normalize(vmin=geo_data['avg_grade'].min(), vmax=geo_data['avg_grade'].max())

# Obtain colors for all points
colors = cmap(norm(geo_data['avg_grade']))

# Create a scatterplot using coordinates, scaled sizes and colors
fig, ax = plt.subplots(figsize=(16, 12))

# Plot the map
gdf.plot(ax=ax)

# Set properties of the scatter plot
ax.scatter(geo_data['lat'], geo_data['long'], s=geo_data['scaled_values'] * 50,
           color=colors, alpha=0.7, edgecolors='k', linewidths=0.5)

# Create a colorbar for visual reference
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.03, fraction=0.05)
cbar.set_label('Average Grade')

# Set the title of the plot
ax.set_title('Map of High Schools in Denmark with Grade Average')

# Adjust subplot parameters to give
plt.tight_layout()

plt.savefig('map.pdf')
plt.show()